In [ ]:
import pandas as pd
import statsmodels.api as sm

I need to add the possibility to add covariates to the untreated twfe--- check
I need to make sure I dont drop the treated but just separate them --- check
I need to construct the function that creates the aggrete parameters using proper weights, create a weights function for this purpose.
I need to make sure the graph for the hours over days is included in the data set 
test the boot error function, and make sure I get the diagonal of the covariance matrix.
I need to create another function that make the testing of parallel trends on the untreated

In [ ]:
def compute_treatment_effects(data, outcome, treatment, unit_id, time_id, covariates=[]):
    # Filter the untreated units for model estimation
    data_untreated = data[data[treatment] == 0]
    
    # Prepare the variables for fixed-effects regression
    features = [outcome, treatment] + covariates
    X = data_untreated.drop(features, axis=1)
    X = pd.get_dummies(X, columns=[unit_id, time_id], drop_first=True)
    
    # Add covariates if any
    if covariates:
        X[covariates] = data_untreated[covariates]
        
    y = data_untreated[outcome]
    
    # Run the two-way fixed effects regression on untreated units
    model = sm.OLS(y, sm.add_constant(X))
    results = model.fit()
    
    # Create a column of predicted values only for treated units
    data_treated = data[data[treatment] == 1]
    X_treated = data_treated.drop(features, axis=1)
    X_treated = pd.get_dummies(X_treated, columns=[unit_id, time_id], drop_first=True)
    
    # Add covariates if any
    if covariates:
        X_treated[covariates] = data_treated[covariates]
        
    data.loc[data[treatment] == 1, 'y_0'] = results.predict(sm.add_constant(X_treated))
    
    # Compute treatment effects
    data['t_effects'] = data[outcome] - data['y_0']
    
    return data

In [ ]:
# Create a toy dataset as an example
import numpy as np
np.random.seed(0)
df = pd.DataFrame({
    'unit_id': np.repeat(range(1, 11), 10),
    'time_id': list(range(1, 11)) * 10,
    'outcome': np.random.normal(0, 1, 100),
    'treatment': np.random.choice([0, 1], 100),
    'covariate1': np.random.normal(0, 1, 100),
    'covariate2': np.random.normal(0, 1, 100),
})

# Apply the function
result_df = compute_treatment_effects(df, 'outcome', 'treatment', 'unit_id', 'time_id', covariates=['covariate1', 'covariate2'])
print(result_df)